In [1]:

from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

import re

import pytz

from time import strptime
import calendar

from datetime import datetime, date
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [2]:
def make_soup(url):
    
    request = requests.get(url)
    if request.status_code == 200:
        soup = BeautifulSoup(request.content, 'html.parser')
    else:
        print('failed: ' + request.status_code)
    
    return soup
    
# url = 'https://www.nuerburgring-langstrecken-serie.de/en/calendar-nurburgring-langstrecken-serie-2023/'

In [3]:
soup = make_soup('https://gotransam.com/events/2023-Trans-Am-Championship/cat/534')


In [4]:
events = soup.find_all('div', {'class':'events-listbox'})

In [5]:
def TA_scrape_locations():
    
    # import re
    
    regex = re.compile("\((.*?)\)")
    locations = []

    events = soup.find_all('div', {'class':'events-listbox'})

    for i in range(len(events)):
        loc = events[i].find('div', {'class':'event-destination'}).text.strip()
        loc = re.sub(regex, '', loc)
        locations.append(loc)
    return locations
    

In [6]:
locations = TA_scrape_locations()
print(locations)

['Sebring International Raceway \nSebring, FL', 'NOLA Motorsports Park \nAvondale, LA', 'Road Atlanta \nBraselton, GA', 'Lime Rock Park \nLakeville, CT', 'Detroit Grand Prix \nDetroit, MI', 'Indianapolis Motor Speedway \nSpeedway, IN', 'Mid-Ohio Sports Car Course \nLexington, OH', 'Road America\nElkhart Lake, WI', 'Big Machine Music City Grand Prix \nNashville, TN', 'Watkins Glen International \nWatkins Glen, NY', 'World Wide Technology Raceway \nMadison, IL', 'VIRginia International Raceway \nAlton, VA', 'Circuit of The Americas \nAustin, TX']


In [7]:
def TA_create_titles():
    
    tit_prt_0 = 'Trans-Am Race R0'
    tit_prt = 'Trans-Am Race R'
    rnd_cnt = 1
    titles = []
    
    events = soup.find_all('div', {'class':'events-listbox'})
    
    for i in range(len(events)):
        if rnd_cnt <10:
            tit = tit_prt_0 + str(rnd_cnt)
        elif rnd_cnt >=10:
            tit = tit_prt + str(rnd_cnt)
        rnd_cnt += 1
        titles.append(tit) 

    return titles

In [8]:
titles = TA_create_titles()
titles

['Trans-Am Race R01',
 'Trans-Am Race R02',
 'Trans-Am Race R03',
 'Trans-Am Race R04',
 'Trans-Am Race R05',
 'Trans-Am Race R06',
 'Trans-Am Race R07',
 'Trans-Am Race R08',
 'Trans-Am Race R09',
 'Trans-Am Race R10',
 'Trans-Am Race R11',
 'Trans-Am Race R12',
 'Trans-Am Race R13']

In [9]:
 
scrp_day = events[0].find('div', {'class':'event-calendar'}).text.split()[1]
scrp_day = scrp_day.split('-')

day_ints = []
for i in scrp_day:
    day_ints.append(int(i))

scrp_days = np.arange(day_ints[0], day_ints[1]+1, 1).tolist()

scrp_days

[23, 24, 25, 26]

In [10]:
len(events)

13

In [11]:
x = events[0].find('div', {'class':'event-calendar'}).text.split()
x

['Feb', '23-26']

In [12]:
scrp_mon = strptime(x[0],'%b').tm_mon #separates the month and changes to string digit '2'
scrp_mon = str(scrp_mon)
scrp_mon

'2'

In [13]:
day_ints = []
scrp_daystr = []

scrp_day = x[1].split('-') #separates the days, changes from '23-26' to ['23', '26']
for i in scrp_day:
    day_ints.append(int(i)) # changes to integers [23, 26] needed for .arrange below
scrp_days = np.arange(day_ints[0], day_ints[1]+1, 1).tolist() #creates a list with all days in the event length [23,24,25,26]
for day in scrp_days:
    scrp_daystr.append(str(day)) # back to str
scrp_daystr

['23', '24', '25', '26']

In [14]:
dates = []
year = '2023'
for day in scrp_daystr:
    assy_date = year + ', ' + day + ', ' + scrp_mon
    dates.append(assy_date)

dates

['2023, 23, 2', '2023, 24, 2', '2023, 25, 2', '2023, 26, 2']

In [15]:
wknd_dates = []
for date in dates:
    date_obj = datetime.strptime(date, '%Y, %d, %m')
    year, month, day = date_obj.year, date_obj.month, date_obj.day
    if calendar.weekday(year, month, day) >= 5:
        wknd_dates.append(date)

In [16]:
wknd_dates

['2023, 25, 2', '2023, 26, 2']

In [17]:

dates = []
wknd_dates = []

year = '2023'

for event in events:
    
    scrp_date = event.find('div', {'class':'event-calendar'}).text.split() #finds full date as ['Feb', '23-26']
    scrp_mon = strptime(scrp_date[0],'%b').tm_mon #separates the month and changes to string digit '2'
    scrp_mon = str(scrp_mon)

    day_ints = []
    scrp_daystr = []
    scrp_day = scrp_date[1].split('-') #separates the days, changes from '23-26' to ['23', '26']
    for i in scrp_day:
        day_ints.append(int(i)) # changes to integers [23, 26] needed for .arrange below
    scrp_days = np.arange(day_ints[0], day_ints[1]+1, 1).tolist() #creates a list with all days in the event length [23,24,25,26]
    for day in scrp_days:
        scrp_daystr.append(str(day)) # back to str

       
    for day in scrp_daystr:
        assy_date = [year, day, scrp_mon]
        dates.append(assy_date)
        
for date in dates:
    date_str = ','.join(date)
    date_obj = datetime.strptime(date_str, '%Y,%d,%m')
    year, month, day = date_obj.year, date_obj.month, date_obj.day
    if calendar.weekday(year, month, day) >= 5:
        wknd_dates.append(date)

In [18]:
wknd_dates

[['2023', '25', '2'],
 ['2023', '26', '2'],
 ['2023', '11', '3'],
 ['2023', '12', '3'],
 ['2023', '25', '3'],
 ['2023', '26', '3'],
 ['2023', '27', '5'],
 ['2023', '28', '5'],
 ['2023', '3', '6'],
 ['2023', '17', '6'],
 ['2023', '18', '6'],
 ['2023', '24', '6'],
 ['2023', '25', '6'],
 ['2023', '8', '7'],
 ['2023', '9', '7'],
 ['2023', '5', '8'],
 ['2023', '9', '9'],
 ['2023', '10', '9'],
 ['2023', '23', '9'],
 ['2023', '24', '9'],
 ['2023', '7', '10'],
 ['2023', '8', '10'],
 ['2023', '4', '11'],
 ['2023', '5', '11']]

In [20]:
decr = []
for event in events:
    try:
        decr = event.find('div', {'class':'tv-broad'}).text.split('MAVTV')
    else:
        print('is this right')

decr

SyntaxError: invalid syntax (1070419849.py, line 5)

In [ ]:
halt

In [ ]:
def scrape_nls(soup):
    
    cln_days = []
    cln_mons = []
    year = 2023
    titles = []
    circuit = []
    
    tr = soup.find_all('tr')
    

## For each round item (tr) find date, trim day to race date only (ie 11-13 becomes just 13, fri-sat becomes just sat)
## store dates in clean_dates in format for iCalendar req's
    for i in range(len(soup.find_all('tr'))): 
        temp_date = soup.find_all('tr')[i].find_all('td')[0].text.strip().split('-')[1].split('.')

        temp_day = temp_date[0]
        if temp_day[0] == '0':
            temp_day = temp_day[1:]
        cln_days.append(temp_day)

        temp_mon = temp_date[1]
        if temp_mon[0] == '0':
            temp_mon = temp_mon[1:]
        cln_mons.append(temp_mon)

        cln_yr = temp_date[2]
        cln_yrs.append(cln_yr)

#     clean_dates = []
#     for yr, mon, day in zip(cln_yrs, cln_mons, cln_days):
#         temp_date_ = yr + ', ' + mon + ', ' + day
#         clean_dates.append(temp_date_)
##


## Pull the event names as description and store to description list
    descriptions = []
    for i in range(len(soup.find_all('tr'))):
        temp_desc = soup.find_all('tr')[i].find_all('td')[1].text.strip()
        descriptions.append(temp_desc)
##

## Make a simple title string for each index.
#  index 0 is preseason test days, mnaully added to list
   
    titles = []
    titles.append('NLS Test Days')
    
    count = 1
    for i in range(len(soup.find_all('tr')[1:9])):
        titles.append('NLS Race R0' + str(count))
        count += 1
#         titles.append(temp_tit)
    
    titles.append('NLS Alternative Date')

        
    return cln_yrs, cln_mons, cln_days, descriptions, titles

In [ ]:
cln_yrs, cln_mons, cln_days, descriptions, titles = scrape_nls(soup)

In [ ]:
cln_yrs

In [ ]:
cln_mons

In [ ]:
cln_days

In [ ]:
descriptions

In [ ]:
titles

In [ ]:
circuit = 'Nürburgring Gesamtrecke' '\n' 'Nürburg, Rhineland-Palatinate, Germany'# 
print(circuit)

In [ ]:
## Creating and Populating the Calendar

In [ ]:
import pytz
import icalendar
from datetime import datetime
from pathlib import Path
import os
from icalendar import Calendar, Event,  vText, vCalAddress

In [ ]:
def new_ical(series_name):
    
   
    series_name = Calendar()
    series_name.add('prodid', '-//My calendar product//example.com//')
    series_name.add('version', '2.0')
    
    return series_name



In [ ]:
NLS = None
NLS = new_ical(NLS)
NLS

In [ ]:


def make_events(cal):
    uid = 110
    circuit = 'Nürburgring Gesamtrecke' '\n' 'Nürburg, Rhineland-Palatinate, Germany'

    for title, description, year, month, day in zip(titles, descriptions, cln_yrs, cln_mons, cln_days):
        ievent = Event()
        ievent.add('summary', title) #Title of the event
        ievent.add('description', description)     #Description of event
        ievent.add('dtstart', datetime(int(year), int(month), int(day), 10, 0, 0, tzinfo=pytz.timezone('utc')))
        ievent.add('dtend', datetime(int(year), int(month), int(day), 14, 0, 0, tzinfo=pytz.timezone('utc')))
        ievent.add('dtstamp', datetime(2023, 3, 17, 0, 10, 0, tzinfo=pytz.timezone('utc')))
        ievent.add('location', vText(circuit))
        ievent.add('uid', uid)
        uid += 1
        cal.add_component(ievent)
        
    return cal
    

In [ ]:
NLS = make_events(NLS)
# NLS

In [ ]:
directory = str(Path(r'C:\Users\chris\Documents\Calendars')) + "/"
print("ics file will be generated at ", directory)
f = open(os.path.join(directory, 'NLS 2023.ics'), 'wb')
f.write(NLS.to_ical())
f.close()
